1\. Write a function that converts number representation (bin<->dec<->hex)

In [4]:
def conv(A):
    b = [A]
    a_bin = bin(A)
    b.append(a_bin)
    a_hex = hex(A)
    b.append(a_hex)
    return b
print (conv(41))

[41, '0b101001', '0x29']


In [9]:
def conv2(A):
    print('The number',A,'in binary representations is:',bin(A),'in hexadecimal representations is:',hex(A))
    return
conv2(41)

The number 41 in binary representations is: 0b101001 in hexadecimal representations is: 0x29


2\. Write a function that converts a 32 bit word into a single precision floating point (i.e. interprets the various bits as sign, mantissa and exponent)

In [13]:
def convert_to_float(x):
    
    print(len(x))
    
    exp = 0
    mantissa = 1
    bias = 127
    s = x[0]
    
    for i in range(1,9):
        exp += int(x[i])*2**(8-i)
    print('exp =',exp)
        
    f = x[9:32]
    for i in range(len(f)):
        mantissa += int(f[i])*2**(-i-1)
    print('Mantissa =',mantissa)
    
    n = mantissa*2**(exp-bias)
    
    if int(s)==0:
        print('Il numero float è:',n)
    if int(s)==1:
        print('Il numero float è:',n)
    
    return

convert_to_float("00111111100000000000000000000000")

32
exp = 127
Mantissa = 1.0
Il numero float è: 1.0


3\. Write a program to determine the underflow and overflow limits (within a factor of 2) for python on your computer. 

**Tips**: define two variables inizialized to 1 and halve/double them enough time to exceed the under/over-flow limits  

In [4]:
import sys
print (sys.float_info)

under=1
over=1

while (over < 1.7976931348623157e+308):
        over *= 2
while (under > 2.2250738585072014e-308):      
        under /= 2     
    
print(over) 
print(under)

sys.float_info(max=1.7976931348623157e+308, max_exp=1024, max_10_exp=308, min=2.2250738585072014e-308, min_exp=-1021, min_10_exp=-307, dig=15, mant_dig=53, epsilon=2.220446049250313e-16, radix=2, rounds=1)
179769313486231590772930519078902473361797697894230657273430081157732675805500963132708477322407536021120113879871393357658789768814416622492847430639474124377767893424865485276302219601246094119453082952085005768838150682342462881473913110540827237163350510684586298239947245938479716304835356329624224137216
2.2250738585072014e-308


4\. Write a program to determine the machine precision

**Tips**: define a new variable by adding a smaller and smaller value (proceeding similarly to prob. 2) to an original variable and check the point where the two are the same 

5\. Write a function that takes in input three parameters $a$, $b$ and $c$ and prints out the two solutions to the quadratic equation $ax^2+bx+c=0$ using the standard formula:
$$
x=\frac{-b\pm\sqrt{b^2-4ac}}{2a}
$$

(a) use the program to compute the solution for $a=0.001$, $b=1000$ and $c=0.001$

(b) re-express the standard solution formula by multiplying top and bottom by $-b\mp\sqrt{b^2-4ac}$ and again find the solution for $a=0.001$, $b=1000$ and $c=0.001$. How does it compare with what previously obtained? Why?

(c) write a function that compute the roots of a quadratic equation accurately in all cases

6\. Write a program that implements the function $f(x)=x(x−1)$

(a) Calculate the derivative of the function at the point $x = 1$ using the derivative definition:

$$
\frac{{\rm d}f}{{\rm d}x} = \lim_{\delta\to0} \frac{f(x+\delta)-f(x)}{\delta}
$$

with $\delta = 10^{−2}$. Calculate the true value of the same derivative analytically and compare with the answer your program gives. The two will not agree perfectly. Why not?

(b) Repeat the calculation for $\delta = 10^{−4}, 10^{−6}, 10^{−8}, 10^{−10}, 10^{−12}$ and $10^{−14}$. How does the accuracy scales with $\delta$?

7\. Consider the integral of the semicircle of radius 1:
$$
I=\int_{-1}^{1} \sqrt(1-x^2) {\rm d}x
$$
which it's known to be $I=\frac{\pi}{2}=1.57079632679...$.
Alternatively we can use the Riemann definition of the integral:
$$
I=\lim_{N\to\infty} \sum_{k=1}^{N} h y_k 
$$

with $h=2/N$ the width of each of the $N$ slices the domain is divided into and where
$y_k$ is the value of the function at the $k-$th slice.

(a) Write a programe to compute the integral with $N=100$. How does the result compares to the true value?

(b) How much can $N$ be increased if the computation needs to be run in less than a second? What is the gain in running it for 1 minute? 
